In [1]:

%matplotlib notebook

# Dependencies
import json
import pprint
import requests
#import sys
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

#api keys
from config import API_Key


In [4]:
url = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'Bearer {}'.format(API_Key),}

#define some business search variables
bus_term = 'restaurants'
bus_location = 'University of Toronto, Toronto, Ontario'
#uoft_lat= 43.6629
#uoft_lon= -79.3957
bus_offset = 950
bus_limit= 50


url_params = {
                'term': bus_term.replace(' ', '+'),
                'location': bus_location.replace(' ', '+'),
                'limit':bus_limit,
                'offset':bus_offset
                
                #'latitude': uoft_lat,
                #'longitude': uoft_lon
                
            }


try:
    business_response = requests.get(url, headers=headers, params=url_params)
    business_data = business_response.json()

    print(business_data)
    
            
        
except:
    print('EXCEPTION')

{'businesses': [{'id': 'AVLj5MSJD1si-Rl6X6qTXQ', 'alias': 'rosewood-chinese-cuisine-toronto-5', 'name': 'Rosewood Chinese Cuisine', 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/b8I4gFzqnvIjJZcvxHGFeA/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/rosewood-chinese-cuisine-toronto-5?adjust_creative=5uhBD6B7pAFlJ3-6q29p5Q&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=5uhBD6B7pAFlJ3-6q29p5Q', 'review_count': 202, 'categories': [{'alias': 'chinese', 'title': 'Chinese'}, {'alias': 'tapasmallplates', 'title': 'Tapas/Small Plates'}], 'rating': 3.0, 'coordinates': {'latitude': 43.6531088463227, 'longitude': -79.3967127799988}, 'transactions': [], 'price': '$$', 'location': {'address1': '463 Dundas Street W', 'address2': '', 'address3': '', 'city': 'Toronto', 'zip_code': 'M5T 1G8', 'country': 'CA', 'state': 'ON', 'display_address': ['463 Dundas Street W', 'Toronto, ON M5T 1G8', 'Canada']}, 'phone': '+14165939998', 'display_phone': '+1 416-593-9998',

In [5]:
restaurants = []
rest_info = {}

for restaurant in business_data["businesses"]:
    
    rest_name = restaurant["name"]
    
    #account for the number of possible categories, we only want a max of 2
    numlen = len(restaurant["categories"])
    #print(numlen)
    if numlen <1:
        rest_type = "n/a"
        rest_subtype = "n/a"
    elif numlen == 1:
        rest_type = restaurant["categories"][0]["title"]
        rest_subtype = "n/a"
    elif numlen > 1:
        rest_type = restaurant["categories"][0]["title"]
        rest_subtype = restaurant["categories"][1]["title"]
    else:
        rest_type = "n/a"
        rest_subtype = "n/a"
    
    rest_lat = restaurant["coordinates"]["latitude"]
    rest_lng = restaurant["coordinates"]["longitude"]
    
    rest_distance = restaurant["distance"]   
    
     
    rest_id = restaurant["id"]
    rest_addr1 = restaurant["location"]["address1"]
    rest_addr2 = restaurant["location"]["address2"]
    rest_city = restaurant["location"]["city"]
    rest_country = restaurant["location"]["country"]
    rest_state = restaurant["location"]["state"]
    rest_zip = restaurant["location"]["zip_code"]
    
    #rest_price = restaurant["price"]
    try:
        rest_price = restaurant["price"]
        rest_price_value = rest_price.count("$")
        
        
    except:
        #print('NO $')
        rest_price = 'NO $'
        rest_price_value = 0
    
    rest_rating = restaurant["rating"]
    rest_review = restaurant["review_count"]
    
    rest_info = {"yelpid":rest_id
                ,"name":rest_name
                ,"lat":rest_lat
                ,"lng":rest_lng
                ,"distance miles": rest_distance
                ,"address1":rest_addr1
                ,"address2":rest_addr2
                ,"city":rest_city
                ,"state":rest_state
                ,"zip":rest_zip
                ,"price rating": rest_price
                ,"price value" : rest_price_value
                ,"review rating":rest_rating
                ,"review count": rest_review
                ,"type":rest_type
                ,"sub type":rest_subtype
                }
    restaurants.append(rest_info)
    
print(restaurants)

[{'yelpid': 'AVLj5MSJD1si-Rl6X6qTXQ', 'name': 'Rosewood Chinese Cuisine', 'lat': 43.6531088463227, 'lng': -79.3967127799988, 'distance miles': 1133.4805209101783, 'address1': '463 Dundas Street W', 'address2': '', 'city': 'Toronto', 'state': 'ON', 'zip': 'M5T 1G8', 'price rating': '$$', 'price value': 2, 'review rating': 3.0, 'review count': 202, 'type': 'Chinese', 'sub type': 'Tapas/Small Plates'}, {'yelpid': 'xdfGzrJ1mx3mBx6m14Ebhw', 'name': 'Goldstone Noodle Restaurant', 'lat': 43.6521515722158, 'lng': -79.3979640306885, 'distance miles': 1250.3036856452748, 'address1': '266 Spadina Avenue', 'address2': '', 'city': 'Toronto', 'state': 'ON', 'zip': 'M5T 2E4', 'price rating': '$', 'price value': 1, 'review rating': 3.0, 'review count': 114, 'type': 'Chinese', 'sub type': 'Noodles'}, {'yelpid': 'HSPMT3m9OOD4ypMO4wwOdg', 'name': 'Fresh On Front', 'lat': 43.648266, 'lng': -79.374338, 'distance miles': 2395.92104696469, 'address1': '47 Front Street E', 'address2': None, 'city': 'Toronto',

In [8]:
#build the dataframe 
restaurant_df = pd.DataFrame(restaurants)

#reorder the columns
restaurant_df = restaurant_df[['yelpid','name','lat','lng','distance miles'
                               ,'price rating','price value','review count', 'review rating'
                              ,'type','sub type','address1','city','state','zip']]
#.set_index('id',[0])
restaurant_df = restaurant_df.reset_index()
restaurant_df



,index,yelpid,name,lat,lng,distance miles,price rating,price value,review count,review rating,type,sub type,address1,city,state,zip
0,0,AVLj5MSJD1si-Rl6X6qTXQ,Rosewood Chinese Cuisine,43.653109,-79.396713,1133.480521,$$,2,202,3.0,Chinese,Tapas/Small Plates,463 Dundas Street W,Toronto,ON,M5T 1G8
1,1,xdfGzrJ1mx3mBx6m14Ebhw,Goldstone Noodle Restaurant,43.652152,-79.397964,1250.303686,$,1,114,3.0,Chinese,Noodles,266 Spadina Avenue,Toronto,ON,M5T 2E4
2,2,HSPMT3m9OOD4ypMO4wwOdg,Fresh On Front,43.648266,-79.374338,2395.921047,$$$,3,35,4.0,Vegetarian,n/a,47 Front Street E,Toronto,ON,M5E 1B3
3,3,QrTvQw_aZOLk_8FZ8tQ_-w,The Bier Markt,43.644740,-79.399920,2076.896788,$$,2,184,3.0,Bars,"Beer, Wine & Spirits",600 King St West,Toronto,ON,M5V 1M3
4,4,XudwKA_sHWiFOxaHLbnfgA,Green Gene,43.655350,-79.384400,1263.814537,NO $,0,1,4.0,Taiwanese,n/a,121 Dundas Street W,Toronto,ON,M5G
5,5,eW5XvdP3ApHx9v76eYCffw,One Pear Cafe,43.654820,-79.387270,1157.689378,NO $,0,37,2.5,Cafes,n/a,201 Dundas Street W,Toronto,ON,M5G 1C8
6,6,98V9R-7YT20KmcmJyMiJmg,Il Fornello,43.647570,-79.386840,1888.932884,$$,2,89,3.0,Italian,n/a,214 King Street W,Toronto,ON,M5H 3S6
7,7,8BT-nH2aWf6nyIdK797OHA,Avant Gout Restaurant,43.678640,-79.390450,1757.678362,NO $,0,14,3.5,French,n/a,1108 Yonge Street,Toronto,ON,M4W 2L6
8,8,D87tFO-R6U8f4-e0r4Zq_g,Yorkville Crepes,43.672193,-79.387866,1174.948256,$$,2,158,3.5,Creperies,Cafes,18 Yorkville Avenue,Toronto,ON,M4W 3Y8
9,9,WmuoWetYC-4U2-V4taC6Wg,Yueh Tung Restaurant,43.655088,-79.385179,1243.319927,$$,2,176,3.5,Hakka,Breakfast & Brunch,126 Elizabeth Street,Toronto,ON,M5G 1P5


In [9]:
restaurant_df.to_csv("UofT_NightOuts.csv")

In [7]:
#setup some variables for the reviews
yelp_reviews = []
reviews_info = {}
yelp_id=[]
for index, row in restaurant_df.iterrows():
    yid = row[1]
    #print(f'{index}: {yid}')
    #restaurant name
    yname = row[2]
    yelp_id.append(yid)
    
    
for i in yelp_id:
        base_url = 'https://api.yelp.com/v3/businesses/'
        review_url= base_url+i+'/reviews'
        review_headers = {'Authorization': 'Bearer {}'.format(API_Key),}
        
        try:
            review_response = requests.get(review_url, headers=review_headers)
            review_responsetype = review_response.status_code
            review_data = review_response.json()
            
        except:      
            print('EXCEPTION')
print(review_data)

for review in review_data["reviews"]:
        
        #the restaurant id we will be using to join on later
        rest_id = yid
        rest_name= yname
        ls_review_id = review["id"]
        ls_review_rating = review["rating"]
        ls_review_text = review["text"]
        ls_review_timestamp = review["time_created"]
        
        #add yelp review name and review id
        ls_review_name = review["user"]["name"]
        ls_review_reviewerid = review["user"]["id"]
        
        
        #check to see if we have text to parse
        #print(ls_review_text)
        
        #--->VADER
        
#         #run vader analysis on each text
#         results = analyzer.polarity_scores(ls_review_text)
#         com = results["compound"]
#         pos = results["pos"]
#         neg = results["neg"]
#         neu = results["neu"]
       
#         #add each value to each list
#         com_ls.append(com)
#         pos_ls.append(pos)
#         neg_ls.append(neg)
#         neu_ls.append(neu)
#         #--------END VADER
        

        reviews_info = {"yelpid":rest_id
                        ,"name": rest_name
                        ,"reviewid": ls_review_id
                        ,"review text": ls_review_text
                        ,"timestamp": ls_review_timestamp
                        ,"reviewer name": ls_review_name
                        ,"reviewer id": ls_review_reviewerid
 #                       ,"compound" : com
#                         ,"positive" : pos
#                         ,"negative" : neg
#                         ,"neutral" : neu
                       }
        yelp_reviews.append(reviews_info)

reviews_df = pd.DataFrame(yelp_reviews)
reviews_df.head()

{'reviews': [{'id': '7ZBbshcb3VMqfwmkNfuHZQ', 'url': 'https://www.yelp.com/biz/smokes-weinerie-toronto?adjust_creative=5uhBD6B7pAFlJ3-6q29p5Q&hrid=7ZBbshcb3VMqfwmkNfuHZQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=5uhBD6B7pAFlJ3-6q29p5Q', 'text': 'Not sure why anyone would go here when Clubhouse is next door. Unfortunately for me, it was unexpectedly closed so I had to settle for some dressed up...', 'rating': 1, 'time_created': '2018-12-27 13:09:19', 'user': {'id': 'IN1YSPjd7-5QthMM7EYyFA', 'profile_url': 'https://www.yelp.com/user_details?userid=IN1YSPjd7-5QthMM7EYyFA', 'image_url': None, 'name': 'Armen A.'}}, {'id': '93_HDm3TOrDYcrPIphNmmQ', 'url': 'https://www.yelp.com/biz/smokes-weinerie-toronto?adjust_creative=5uhBD6B7pAFlJ3-6q29p5Q&hrid=93_HDm3TOrDYcrPIphNmmQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=5uhBD6B7pAFlJ3-6q29p5Q', 'text': "Their specialty does not come in snack size. WHY. I can't finish a regular. \n\nI never rea

,name,review text,reviewer id,reviewer name,reviewid,timestamp,yelpid
0,Smoke's Weinerie,Not sure why anyone would go here when Clubhou...,IN1YSPjd7-5QthMM7EYyFA,Armen A.,7ZBbshcb3VMqfwmkNfuHZQ,2018-12-27 13:09:19,7T1XVTSocHcOu2H5v2Iqog
1,Smoke's Weinerie,Their specialty does not come in snack size. W...,UXpTrzITc0yg08sXftsa-w,Irene T.,93_HDm3TOrDYcrPIphNmmQ,2016-08-18 06:52:41,7T1XVTSocHcOu2H5v2Iqog
2,Smoke's Weinerie,Apparently this place is much better at 4:00a ...,wu8Qh6QzgdoFuXux59p8QA,Jason W.,Em8ADPYe150y-weAFpHfwQ,2017-10-10 20:31:50,7T1XVTSocHcOu2H5v2Iqog
